# Using foambryo

In this notebook, we'll see:
- How to install foambryo,
- What are the inputs of foambryo,
- How to compute relative tensions and pressures insides cells,
- How to plot these forces using the foambryo viewer.

# 0) Installation

Foambryo is available on pyPI such that you can just use pip !

- if you don't need any visualization, use `pip install foambryo`.
- if you want to use the foambryo viewer to visualize the forces on a 3D mesh, use `pip install "foambryo[viewing]"`.

In this notebook we'll show how to visualize the results, so we choose the later option.

In [ ]:
! pip install "foambryo[viewing]"

In [1]:
!which python

/Users/kyamauch/Documents/miniforge3/envs/force-inference/bin/python


# 1) Prepare an inference mesh

Foambryo actually computes tensions and pressures from a datastructure called a DCEL mesh. Foambryo can create such a mesh from 2 sources:
- an instance segmentation mask, using the Delaunay-Watershed-3D algorithm,
- a multimaterial mesh.

We provide easy functions to have a mesh ready for force inference:

In [2]:
import skimage.io as io
from dw3d.io import load_rec
from foambryo import dcel_mesh_from_file, dcel_mesh_from_multimaterial_mesh, dcel_mesh_from_segmentation_mask

# 1) From a segmentation mask
image_path = "../data/Segmentations/Benchmark_labels.tif"
segmentation_mask = io.imread(image_path)  # open segmentation image.
# If here you have a problem, make sure you have git lfs installed and run git lfs pull to really download this large file.

dcel_mesh_1 = dcel_mesh_from_segmentation_mask(segmentation_mask, min_distance=3, print_info=True)  # DCEL mesh ready for force inference !
# Note: mesh reconstruction from a segmentation mask can take some time.

# 2) From a multimaterial mesh file (3D triangle mesh + material data)
dcel_mesh_2 = dcel_mesh_from_file("../data/Meshes/simple_mesh.vtk")

# 3) From an opened multimaterial mesh (3 arrays: points, triangles, labels/materials)
# Here we open a mesh to simulate the situation:
points, triangles, labels = load_rec("../data/Meshes/Benchmark_original_mesh.rec")
dcel_mesh_3 = dcel_mesh_from_multimaterial_mesh(points, triangles, labels)

Computing EDT ...
EDT computed in  0.72
Searching local extremas ...
Number of local minimas : 6358
Number of local maxes : 153
Delaunay Tesselation built in 0.27 seconds
Graph build in  0.693
Watershed done in 0.253 seconds.


# 2) Force Inference

Now that we have a DCEL mesh, we can easily compute relative pressures between cells and relative tensions at interfaces between cells (or with the exterior region).

In [3]:
from foambryo import infer_forces

# select mesh
dcel_mesh = dcel_mesh_1

# infer both tensions and pressures
tensions, pressures = infer_forces(dcel_mesh)

The outputs are simple data structures:

In [4]:
# pressures is a dictionary that maps cell index to pressure force. 0 is the exterior.
print(f"{pressures=}")

# tensions is a dictionary that maps cells interfaces (cell index 1, cell index 2) to tension force. 0 is the exterior.
print(f"{tensions=}")

pressures={0: 0, np.int64(1): np.float64(0.017742905051825082), np.int64(2): np.float64(0.03927119387118719), np.int64(3): np.float64(0.03690598205451832), np.int64(4): np.float64(0.04492606222374091), np.int64(5): np.float64(0.044045323227494965), np.int64(6): np.float64(0.033135580447285165)}
tensions={(np.int64(0), np.int64(1)): np.float64(0.47342023816347106), (np.int64(0), np.int64(2)): np.float64(0.8753760645516024), (np.int64(0), np.int64(3)): np.float64(0.867027772295751), (np.int64(0), np.int64(4)): np.float64(1.0036769829654757), (np.int64(0), np.int64(5)): np.float64(0.9725211571313266), (np.int64(0), np.int64(6)): np.float64(0.8276201845764745), (np.int64(1), np.int64(2)): np.float64(0.6857833446078545), (np.int64(1), np.int64(3)): np.float64(0.861349824943724), (np.int64(1), np.int64(4)): np.float64(1.002242342907234), (np.int64(1), np.int64(5)): np.float64(1.0469225590458329), (np.int64(1), np.int64(6)): np.float64(0.8058025358044713), (np.int64(2), np.int64(3)): np.float

You can also compute tensions only, but for the pressures, you need to have first computed the tensions:

In [5]:
from foambryo import infer_pressures, infer_tensions

tensions = infer_tensions(dcel_mesh)

pressures = infer_pressures(dcel_mesh, tensions)

In [6]:
tensions

{(np.int64(0), np.int64(1)): np.float64(0.47342023816347106),
 (np.int64(0), np.int64(2)): np.float64(0.8753760645516024),
 (np.int64(0), np.int64(3)): np.float64(0.867027772295751),
 (np.int64(0), np.int64(4)): np.float64(1.0036769829654757),
 (np.int64(0), np.int64(5)): np.float64(0.9725211571313266),
 (np.int64(0), np.int64(6)): np.float64(0.8276201845764745),
 (np.int64(1), np.int64(2)): np.float64(0.6857833446078545),
 (np.int64(1), np.int64(3)): np.float64(0.861349824943724),
 (np.int64(1), np.int64(4)): np.float64(1.002242342907234),
 (np.int64(1), np.int64(5)): np.float64(1.0469225590458329),
 (np.int64(1), np.int64(6)): np.float64(0.8058025358044713),
 (np.int64(2), np.int64(3)): np.float64(1.0464774350284478),
 (np.int64(2), np.int64(4)): np.float64(1.4698428080484314),
 (np.int64(2), np.int64(6)): np.float64(1.1381418534665038),
 (np.int64(3), np.int64(6)): np.float64(0.5625781065629598),
 (np.int64(4), np.int64(5)): np.float64(1.7008682638227197),
 (np.int64(4), np.int64(6)

An important thing to understand is that we compute relative tensions and pressures. As such, when computing pressures, you need to give a base pressure (the pressure of the exterior, the medium) ; when computing tensions, you need to give a mean tension.

By default, the base pressure is 0 while the mean tension is 1. You can always set it as desired:

In [7]:
# previous lines where equivalent to these:
tensions, pressures = infer_forces(dcel_mesh, mean_tension=1, base_pressure=0)
tensions = infer_tensions(dcel_mesh, mean_tension=1)
pressures = infer_pressures(dcel_mesh, tensions, base_pressure=0)

Finally, foambryo implements several strategies to compute tensions and pressures. The default strategies give good results, but you can explore the other strategies if you want.

In [8]:
from foambryo import PressureComputationMethod, TensionComputationMethod

print(f"Available pressure computation methods: {[pressure_method.name for pressure_method in PressureComputationMethod]}.")

print(f"Available tension computation methods: {[tension_method.name for tension_method in TensionComputationMethod]}.")

# previous lines where equivalent to these:
tensions, pressures = infer_forces(dcel_mesh, mode_tension=TensionComputationMethod.YoungDupre, mode_pressure=PressureComputationMethod.Variational)
tensions = infer_tensions(dcel_mesh, mode=TensionComputationMethod.YoungDupre)
pressures = infer_pressures(dcel_mesh, tensions, mode=PressureComputationMethod.Variational)

Available pressure computation methods: ['Variational', 'Laplace', 'WeightedLaplace'].
Available tension computation methods: ['YoungDupre', 'ProjectionYoungDupre', 'Equilibrium', 'Cotan', 'InvCotan', 'Lami', 'InvLami', 'LogLami', 'Variational'].


# 3) Plotting forces

Now that we have tensions and pressures, foambryo can also plot the mesh with the values.

To plot pressures:

In [9]:
from foambryo.viewing import plot_pressure_inference

# Note: only the mesh in necessary. If pressures are not provided, they'll be computed.
plot_pressure_inference(dcel_mesh, pressures)

Extremas of pressures plotted:  0 0.04492606222374091
[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 89.4


To plot tensions:

In [8]:
from foambryo.viewing import plot_tension_inference

# Note: only the mesh in necessary. If tensions are not provided, they'll be computed.
# For tensions, it's better to scatter the cells
plot_tension_inference(dcel_mesh, tensions, scattered=True, scatter_coeff=1)

Extremas of the Surface Tensions plotted :  0.5492044263030357 1.5338733278807724


To plot both on the same viewer:

In [ ]:
from foambryo.viewing import plot_force_inference

plot_force_inference(dcel_mesh, tensions, pressures)

Many more data can be plotted, such s the curvature, the area and volume derivatives...

In [ ]:
plot_force_inference(dcel_mesh, tensions, pressures, scalar_quantities=True)

Plotting functions have many interesting parameters, don't hesitate to play with them !